In [5]:
import sys
sys.path.append('../')

In [14]:
from IPython.display import display, HTML
from tft.data import DataLoader

In [7]:
dt = DataLoader(data_dir='/storage/samba/work/gnzh/mortdogged/analysis/data').load_all()

In [8]:
def winrate(x):
    return x.where(x == True).count() / len(x)

def true_count(x):
    return x.where(x == True).count()

def total_perc(total):
    return lambda x: len(x)/total

def winrate_for(df, group: str, cutoff = 1, perc_cutoff = 0.02):
    df['winrate'] = df['placement'] <= cutoff
    df['win'] = df['placement'] <= cutoff
    df['loss'] = df['placement'] >= cutoff
    df['total_perc'] = 1
    cnt = len(df)
    wr = df.groupby(group).agg({
        'winrate': winrate,
        'win': true_count, 'loss': true_count,
        'total_perc': total_perc(cnt),
    })
    return wr[wr['total_perc'] > perc_cutoff].sort_values(by=['winrate'], ascending=False)
    

In [21]:
player_winrate = winrate_for(dt.participants, 'puuid', perc_cutoff = 0.00001)

In [22]:
mn = player_winrate['winrate'].mean()
HTML(f"Mean player winrate <strong>{round(mn * 100, 2)}%</strong>")

In [9]:
winrate_for(dt.units, 'character_id')

,winrate,win,loss,total_perc
character_id,,,,
TFT7_Zoe,0.215638,1481,6868,0.035295
TFT7_Bard,0.214431,1893,8828,0.045368
TFT7_Yasuo,0.203772,1599,7847,0.040326
TFT7_Soraka,0.197198,1154,5852,0.030074
TFT7_Ornn,0.149044,998,6696,0.034411
TFT7_Heimerdinger,0.136102,898,6598,0.033908
TFT7_Sylas,0.134974,1289,9550,0.049078
TFT7_Lulu,0.132140,1712,12956,0.066582
TFT7_TrainerDragon,0.129543,1023,7897,0.040583


In [10]:
winrate_for(dt.items, 'item')

,winrate,win,loss,total_perc
item,,,,
TFT_Item_SpearOfShojin,0.169572,1138,6711,0.023946
TFT_Item_IonicSpark,0.168320,967,5745,0.020500
TFT_Item_GuardianAngel,0.167840,1001,5964,0.021281
TFT_Item_TitanicHydra,0.167380,978,5843,0.020849
TFT_Item_ZekesHerald,0.166667,1188,7128,0.025434
TFT_Item_ArchangelsStaff,0.157013,1705,10859,0.038748
TFT_Item_Morellonomicon,0.155633,938,6027,0.021506
TFT_Item_StatikkShiv,0.152932,1338,8749,0.031219
TFT_Item_ThiefsGloves,0.148606,1226,8250,0.029438


In [11]:
winrate_for(dt.augments, 'augment')

,winrate,win,loss,total_perc
augment,,,,
TFT6_Augment_Weakspot,0.142518,240,1684,0.022437
TFT7_Augment_BandOfThieves1,0.139757,219,1567,0.020878
TFT6_Augment_Ascension,0.137274,288,2098,0.027953
TFT6_Augment_CelestialBlessing2,0.132450,280,2114,0.028166
TFT7_Augment_UrfsGrabBag1,0.130715,243,1859,0.024769
TFT6_Augment_PortableForge,0.130549,347,2658,0.035415
TFT6_Augment_ThrillOfTheHunt1,0.123762,200,1616,0.021531
TFT6_Augment_ComponentGrabBag,0.112966,203,1797,0.023943
TFT6_Augment_SecondWind1,0.112102,176,1570,0.020918


In [12]:
star3_df = dt.units.loc[:][dt.units['tier'] == 3]
winrate_for(star3_df, 'character_id')

,winrate,win,loss,total_perc
character_id,,,,
TFT7_Nunu,0.223796,79,353,0.027202
TFT7_Anivia,0.184211,63,342,0.026354
TFT7_Volibear,0.172330,71,412,0.031748
TFT7_Diana,0.157895,57,361,0.027818
TFT7_Olaf,0.153664,65,423,0.032596
TFT7_Sett,0.152344,78,512,0.039454
TFT7_Vladimir,0.136364,42,308,0.023734
TFT7_Illaoi,0.114787,170,1481,0.114125
TFT7_Nami,0.112000,196,1750,0.134854


In [13]:
winrate_for(dt.items, ['character_id', 'item'], perc_cutoff = 0.005)

winrate   win  loss  \
character_id       item                                                
TFT7_AoShin        TFT_Item_ArchangelsStaff     0.251844   478  1898   
TFT7_Yasuo         TFT_Item_Bloodthirster       0.222680   324  1455   
TFT7_Lulu          TFT_Item_Chalice             0.184151   323  1754   
                   TFT_Item_ZekesHerald         0.170292   321  1885   
TFT7_Sylas         TFT_Item_TitanicHydra        0.170249   301  1768   
TFT7_DragonGold    TFT_Item_GargoyleStoneplate  0.167121   245  1466   
TFT7_Neeko         TFT_Item_WarmogsArmor        0.153674   274  1783   
TFT7_Sylas         TFT_Item_WarmogsArmor        0.153486   295  1922   
                   TFT_Item_RedBuff             0.152503   454  2977   
                   TFT_Item_GargoyleStoneplate  0.141938   353  2487   
TFT7_Xayah         TFT_Item_GuinsoosRageblade   0.132321   244  1844   
TFT7_TrainerDragon TFT7_Item_TrainerSnax        0.129543  1023  7897   
TFT7_DragonPurple  TFT_Item_TitansResolve       0.127533   321  2517   
TFT7_Neeko         TFT_Item_RedBuff             0.126933   238  1875   
                   TFT_Item_GargoyleStoneplate  0.126649   240  1895   
TFT7_DragonBlue    TFT_Item_GuinsoosRageblade   0.126406   236  1867   
TFT7_DragonPurple  TFT_Item_Bloodthirster       0.117609   362  3078   
TFT7_DragonBlue    TFT_Item_ArchangelsStaff     0.109375   196  1792   
TFT7_DragonGreen   TFT_Item_Bloodthirster       0.099624   212  2128   
TFT7_Anivia        TFT_Item_ArchangelsStaff     0.098886   142  1436   
TFT7_Corki         TFT_Item_GuinsoosRageblade   0.097152   174  1791   
TFT7_DragonGreen   TFT_Item_TitansResolve       0.094183   204  2166   

                                                total_perc  
character_id       item                                     
TFT7_AoShin        TFT_Item_ArchangelsStaff       0.006773  
TFT7_Yasuo         TFT_Item_Bloodthirster         0.005192  
TFT7_Lulu          TFT_Item_Chalice               0.006259  
                   TFT_Item_ZekesHerald           0.006726  
TFT7_Sylas         TFT_Item_TitanicHydra          0.006309  
TFT7_DragonGold    TFT_Item_GargoyleStoneplate    0.005231  
TFT7_Neeko         TFT_Item_WarmogsArmor          0.006362  
TFT7_Sylas         TFT_Item_WarmogsArmor          0.006858  
                   TFT_Item_RedBuff               0.010623  
                   TFT_Item_GargoyleStoneplate    0.008874  
TFT7_Xayah         TFT_Item_GuinsoosRageblade     0.006580  
TFT7_TrainerDragon TFT7_Item_TrainerSnax          0.028178  
TFT7_DragonPurple  TFT_Item_TitansResolve         0.008981  
TFT7_Neeko         TFT_Item_RedBuff               0.006690  
                   TFT_Item_GargoyleStoneplate    0.006762  
TFT7_DragonBlue    TFT_Item_GuinsoosRageblade     0.006662  
TFT7_DragonPurple  TFT_Item_Bloodthirster         0.010983  
TFT7_DragonBlue    TFT_Item_ArchangelsStaff       0.006394  
TFT7_DragonGreen   TFT_Item_Bloodthirster         0.007593  
TFT7_Anivia        TFT_Item_ArchangelsStaff       0.005124  
TFT7_Corki         TFT_Item_GuinsoosRageblade     0.006391  
TFT7_DragonGreen   TFT_Item_TitansResolve         0.007729